In [1]:

from battle import *
from numpy.random import random, randint
from termcolor import colored
# (position=[0,0])
chessCanBeUsed = [ant,
                  llama,
                  wolf, 
                  ladybug, 
                  bee, 
                  swallower, 
                  sea_hedgehog, 
                  heal_deer, 
                  hippo, 
                  bear, 
                  butterfly, 
                  fireworm, 
                  mantis, 
                  scorpion, 
                  anglerfish, 
                  tiger, 
                  shark, 
                  spider, 
                  rabbit, 
                #   anglerfish(position=[0,0])
                  
                  ]
cost1Card = [ant, littleUglyFish, rabbit]
cost2Card = [llama, wolf, ladybug, bee, swallow, sea_hedgehog]
unavail = {electric_eel,monoceros,turtle,elephant,unicorn_b,littleUglyFish,monkey} # octopus
# 初始棋子
redTeam = []
blueTeam = []
class Player:
  # id = 0
  def __init__(self, id) -> None:
    # self.id = Player.id
    # Player.id += 1
    self.id = id
    self.round:int = 1
    self.chesses: dict[int, chessInterface] = {}
    self.chessInHand: list[chessInterface] = [] # max Of 5 cards
    self.chessOnField:list[chessInterface] = []
    self.goldAvail = 1
    self.buffs = []
    self.population = 1
    self.roundToNextPop = 1
    self.goldToNextPop = 1
    self.cost2Chance = 0 # 抽到2星卡的概率
    self.hp = 100

  def __repr__(self) -> str:
    if self.id == 0:
      color = 'red'
    else: 
      color = 'blue'
    return f"{colored('玩家'+str(self.id),color)}"
  
  def printStatus(self) -> None:
    print (f"\n" + \
          f"当前回合 {self.round}, 剩余血量：{self.hp}\n" + \
          f"{self} 现在拥有 {colored(self.population,'green')} 人口，距离下次升级还需要{colored(self.roundToNextPop,'light_cyan')}回合，或者{colored(self.roundToNextPop*self.roundToNextPop,'light_cyan')}金钱；\n" + \
          f"现在拥有的棋子为{self.chesses}\n" + \
          f"备战区棋子：{self.chessInHand}，上场棋子{self.chessOnField}；\n"+ \
          f"当前回合还有可用金钱{colored(self.goldAvail, 'yellow')}；\n" + \
          f"玩家buff有{self.buffs}；\n" + \
          "\n")

  def next_round(self):
    '''每回合开始阶段调用这个方法'''
    self.hp -= 10
    self.round += 1
    self.goldAvail = deepcopy(self.round)
    self.checkPopulationUpdate()      
    self.printStatus()  
  
# 金钱部分
  def reduce_gold(self, howMuch, purpose: str):
    '''花钱的时候调用这个方法'''
    if self.goldAvail - howMuch >= 0:
      self.goldAvail -= howMuch
      print(self,purpose,f"{colored('成功','green')},共花费: ",howMuch," 金币，还剩 ",self.goldAvail," 金币。")
      return True
    else:
      print(self,purpose,f"{colored('失败','red')},需要: ",howMuch,' 金币，现有 ',self.goldAvail,f" 金币，差{howMuch - self.goldAvail}金币。")
      return False
    
  def checkPopulationUpdate(self):
    """检查人口是否上升"""
    self.roundToNextPop -= 1
    self.goldToNextPop -= self.population
    if self.roundToNextPop == 0:
      self.population += 1
      self.roundToNextPop = deepcopy(self.population)
      self.costForNextPop = deepcopy(self.population)
      print(f"玩家{self.id} 现在拥有 {colored(self.population,'green')} 人口，升级至下一人口还需要{colored(self.roundToNextPop,'light_cyan')}回合，或者 {colored(self.roundToNextPop*self.roundToNextPop,'light_cyan')} 金钱；\n")
      
# 人口部分
  def upgrade_pop(self):
    """用金币升级人口"""
    if self.reduce_gold(purpose=f"升级人口", howMuch=self.population):
      self.checkPopulationUpdate()
      
# 棋子部分
  def printChesses(self):
    print(self.chesses)
    
  def get_random_chess_to_draw(self):
    """每回合随机抽卡"""
    cardList = []
    for i in range(3):
      if random() < self.cost2Chance:
        cardList.append( cost2Card[randint(0,len(cost2Card))] )
      else:
        cardList.append( cost1Card[randint(0,len(cost1Card))] )
    print(self,"本次选择有",cardList)
    return cardList
  def redraw(self):
    """重新抽卡"""
    if self.reduce_gold(purpose=f'刷新棋子', howMuch = 3):
      self.get_random_chess_to_draw()
    
  def checkChessID(self, chessID:int) -> chessInterface:
    print(chessID, type(chessID),self.chesses)
    try:
      chess = self.chesses[chessID]
    except:
      raise KeyError(f"{self}没有这张卡")
    return chess

  def get_chess(self,newChess: chessInterface, purchase = True):
    """购买棋子"""
    newChess.team = self.id
    if purchase is True:
      if self.reduce_gold(purpose=f"购买棋子{newChess}", howMuch=newChess.star):
        self.chessInHand.append(newChess)
        self.chesses[newChess.uniqueID] = newChess 
    else:
      print(self,f"通过升级获取了 <{newChess}>")
      self.chessInHand.append(newChess)
      self.chesses[newChess.uniqueID] = newChess 
    self.printStatus()

  def sell_chess(self, chessID: int):
    """出售棋子"""
    chess = self.checkChessID(chessID=chessID)
    if chess.onField:
      self.chessOnField.remove(chess)
      self.goldAvail += chess.star # 玩家在升级四星之后，如果出售棋子只能获得一半的金钱，所以需要提前斟酌自己的阵容
      print(f"{self}:已被卖掉，从chessOnField and chesses中移除,{self}获得金钱{chess.star}，现在闲置金钱{self.goldAvail}")
    else:
      self.chessInHand.remove(chess)
      self.goldAvail += chess.star
      print(f"{self}:已被卖掉，从chessInHand and chesses中移除,{self}获得金钱{chess.star}，现在闲置金钱{self.goldAvail}")
    for (s, c) in self.chesses.items():
      if c == chess:
        break
    del self.chesses[s]
    print(  f"       现在拥有的棋子为{self.chesses}")
    print(  f"       备战区棋子：{self.chessInHand}，\n       上场棋子{self.chessOnField}；")
      
  def hand_to_field(self, chessID: int, position: list[int]):
    """上场棋子"""
    # 判定人口
    if len(self.chessOnField) >= self.population:
      print(f"人口不足，请先下棋子，选项有{self.chessOnField}")
    chess = self.checkChessID(chessID=chessID)
    if chess in self.chessOnField:
      print(f"棋子{chess}已经在场上了，请勿重复上场!")
      return
    self.chessInHand.remove(chess)
    self.chessOnField.append(chess)
    chess.setInitialPosition(position = position)
    chess.onField = True
    print(f"{self}:{chess}上场到位置{colored(str(chess.position), color='magenta')}")
    print(f"       备战区棋子：{self.chessInHand}，\n       上场棋子{self.chessOnField}；")
    
  def field_to_hand(self, chessID: int):
    """下场棋子"""
    chess = self.checkChessID(chessID=chessID)
    if chess in self.chessInHand:
      print(f"棋子{chess}已经在备战区了，不能下场！")
      return
    chess.position = [-1,-1]
    self.chessOnField.remove(chess)
    self.chessInHand.append(chess)
    chess.onField = False
    print(f"{self}:{chess}下场")
    print(f"       备战区棋子：{self.chessInHand}，\n       上场棋子{self.chessOnField}；")
    
  
# 棋子升级
  def first_2_copy(self, chessID: int) -> list[chessInterface]:
    chess = self.checkChessID(chessID=chessID)
    sameChess = []
    for c in self.chesses.values():
      if c.id == chess.id:
        sameChess.append(c)
    if len(sameChess) >= 2:
      return sameChess[:2]
    else:
      return []

  def upgrade_chess(self, chessID: int ):
    chess = self.checkChessID(chessID=chessID)
    sameChessList = self.first_2_copy(chessID=chessID)
    if len(sameChessList) >= 2:
      for c in sameChessList:
        if c.onField:
          self.chessOnField.remove(c)
        else:
          self.chessInHand.remove(c)
        del self.chesses[c.uniqueID]
        c.onField = False
      print(f"  {self}:{chess}升级{colored('成功','green')}，请选择一个 {chess.star + 1} 星棋子")
      
      inp: int = int(input(f"         选项有：{[self.chessName_dict[i] for i in self.upgrade_dict[chess.id]]}"))
      while inp not in self.upgrade_dict[chess.id]:
        inp = input(f"    {chess}不可以升级为{self.chess_dict[inp]}\n    请在{self.upgrade_dict[chess.id]}中选择合理的棋子ID：")
        inp = int(inp)
      self.get_chess(self.chess_dict[inp](),purchase=False)
    else:
      print(f"  {self}:{chess}升级{colored('失败','red')}，请检查棋子持有数量")
        
  upgrade_dict = {1: [4, 5], 2: [6, 7], 3: [8, 9], # 1 star 兔子，蚂蚁，小丑鱼

                  4: [10, 11], 5: [12, 13], # 2 star mammal 4：羊驼， 5:狼 
                  6: [14, 15], 7: [16, 17],  # 2 star insect 6: 瓢虫 ，7: 蜜蜂
                  8: [18, 19], 9: [20, 21], # 2 star marine  8: 食人鱼 9: 海胆

                  10: [23,24], 11: [23,24], # 3 star mammal ranged 麋鹿， 猴
                  12: [23,24], 13: [23,24], # 3 star mammal melee 犀牛， 熊

                  14: [25,26], 15: [25,26], # 3 star insect ranged 蝴蝶， 萤火虫
                  16: [25,26], 17: [25,26], # 3 star insect melee 螳螂， 蝎子
                  
                  18: [27,28], 19: [27,28], # 3 star marine ranged 灯笼鱼， 电鳗
                  20: [27,28], 21: [27,28], 22:[], # 3 star marine ranged 螃蟹，一角鲸，海龟

                  23:[],24:[], # 大象， 老虎
                  25:[], 26:[],  # 独角仙， 蜘蛛
                  27:[], 28:[]} # 章鱼， 鲨鱼
  chess_dict = {1: rabbit, 2:ant, 3:littleUglyFish, 
                
                4:llama, 5:wolf, 
                6:ladybug, 7: bee, 
                8: swallower, 9:sea_hedgehog,  
                
                10: heal_deer, 11: monkey, 
                12: hippo, 13: bear,
                14: butterfly, 15: fireworm,
                16: mantis, 17: scorpion, # 3 star insect melee 螳螂， 蝎子
                  
                18: anglerfish, 19: electric_eel, # 3 star marine ranged 灯笼鱼， 电鳗
                20: crab, 21: monoceros, 22: turtle, # 3 star marine ranged 螃蟹，一角鲸，海龟

                23:elephant,24:tiger, # 大象， 老虎
                25:unicorn_b, 26:spider,  # 独角仙， 蜘蛛
                27:octopus, 28:shark} # 章鱼， 鲨鱼

  chessName_dict = {1: '兔子 id:1', 2:'蚂蚁 id:2', 3:"小丑鱼 id:3", 
                
                4:"羊 id:4", 5:"狼 id:5", 
                6:'瓢虫 id:6', 7: '蜜蜂 id:7', 
                8: '食人鱼 id:8', 9:'河豚 id:9',  
                
                10: '麋鹿 id:10', 11: '猴子 id:11', 
                12: '河马 id:12', 13: '熊 id:13',
                14: '蝴蝶 id:14', 15: '萤火虫 id:15',
                16: '螳螂 id:16', 17: '蝎子 id:17', # 3 star insect melee 螳螂， 蝎子
                  
                18: '灯笼鱼 id:8', 19: '电鳗 id:19', # 3 star marine ranged 灯笼鱼， 电鳗
                20: '螃蟹 id:20', 21: '一角鲸 id:21', 22: '海龟 id:22', # 3 star marine ranged 螃蟹，一角鲸，海龟

                23:'大象 id:23',24:'老虎 id:24', # 大象， 老虎
                25:'独角仙 id:25', 26:'蜘蛛 id:26',  # 独角仙， 蜘蛛
                27:'章鱼 id:27', 28:'鲨鱼 id:28'} # 章鱼， 鲨鱼
  # def get_chess(self,chess:chessInterface):
  #   if chess.id in [1,2,3]:
  #     self.goldAvail -= 1
  #   elif chess.id in [4,5,6,7,8,9]:
  #     self.goldAvail -= 2

  

In [2]:
# 初始化玩家
player1 = Player(0)
player2 = Player(1)

回合1


每人1棋子

In [3]:
player1.printStatus()
player2.printStatus()
player1.get_random_chess_to_draw()
player2.get_random_chess_to_draw()




当前回合 1, 剩余血量：100
玩家0 现在拥有 1 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{}
备战区棋子：[]，上场棋子[]；
当前回合还有可用金钱1；
玩家buff有[]；



当前回合 1, 剩余血量：100
玩家1 现在拥有 1 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{}
备战区棋子：[]，上场棋子[]；
当前回合还有可用金钱1；
玩家buff有[]；


玩家0 本次选择有 [<class 'chess_info.rabbit'>, <class 'chess_info.rabbit'>, <class 'chess_info.ant'>]
玩家1 本次选择有 [<class 'chess_info.littleUglyFish'>, <class 'chess_info.ant'>, <class 'chess_info.ant'>]


[chess_info.littleUglyFish, chess_info.ant, chess_info.ant]

In [4]:
# 选择棋子
player1.get_chess(rabbit())
player2.get_chess(ant())

player1.printChesses()
player2.printChesses()


玩家0 购买棋子兔子1 成功,共花费:  1  金币，还剩  0  金币。
玩家1 购买棋子蚂蚁2 成功,共花费:  1  金币，还剩  0  金币。
{1: 兔子1}
{2: 蚂蚁2}


In [5]:
# 准备阶段
player1Chess = 1
player2Chess = 2

# 上场棋子
player1.hand_to_field(player1Chess, position=[2,2])
player2.hand_to_field(player2Chess, position=[3,2])

# 下场棋子

# player1.field_to_hand(player1Chess)
# player2.field_to_hand(player2Chess)

# player1.printStatus()
# player2.printStatus()


1 <class 'int'> {1: 兔子1}
玩家0:兔子1上场到位置[2, 2]
       备战区棋子：[]，
       上场棋子[兔子1]；
2 <class 'int'> {2: 蚂蚁2}
玩家1:蚂蚁2上场到位置[3, 2]
       备战区棋子：[]，
       上场棋子[蚂蚁2]；


In [6]:
# 战斗阶段
round1Battle = battle()
round1Battle.addRedTeam(redTeam= player1.chessOnField)
round1Battle.addBlueTeam(blueTeam= player2.chessOnField)
round1Battle.battle_with_skills()



[None, None, None, None, None]

[None, None, None, None, None]

[None, None, 兔子1, None, None]

[None, None, 蚂蚁2, None, None]

[None, None, None, None, None]

[None, None, None, None, None]


0.9 <兔子1> 攻击了<蚂蚁2>, 造成了22.10526315789474点伤害，<蚂蚁2>生命值还剩:157.89473684210526 / 180
1.0 <蚂蚁2> 攻击了<兔子1>, 造成了24.539877300613497点伤害，<兔子1>生命值还剩:225.4601226993865 / 250
1.8 <兔子1> 攻击了<蚂蚁2>, 造成了22.10526315789474点伤害，<蚂蚁2>生命值还剩:135.78947368421052 / 180
2.0 <蚂蚁2> 攻击了<兔子1>, 造成了24.539877300613497点伤害，<兔子1>生命值还剩:200.92024539877298 / 250
2.7 <兔子1> 攻击了<蚂蚁2>, 造成了22.10526315789474点伤害，<蚂蚁2>生命值还剩:113.68421052631578 / 180
3.0 <蚂蚁2> 攻击了<兔子1>, 造成了24.539877300613497点伤害，<兔子1>生命值还剩:176.38036809815947 / 250
3.6 <兔子1> 攻击了<蚂蚁2>, 造成了22.10526315789474点伤害，<蚂蚁2>生命值还剩:91.57894736842104 / 180
4.0   兔子1对蚂蚁2使用了*回手掏*,造成100点伤害
4.0   蚂蚁2被【眩晕】了！

    蚂蚁2 已经被打败!

红方获胜!

[None, None, None, None, None]

[None, None, None, None, None]

[None, None, 兔子1, None, None]

[None, None, None, None, None]

[None, None, None, None, None]

[None, None, Non

回合2


In [7]:
player1.next_round()
player2.next_round()

player1.get_random_chess_to_draw()
player2.get_random_chess_to_draw()

玩家0 现在拥有 2 人口，升级至下一人口还需要2回合，或者 4 金钱；


当前回合 2, 剩余血量：90
玩家0 现在拥有 2 人口，距离下次升级还需要2回合，或者4金钱；
现在拥有的棋子为{1: 兔子1}
备战区棋子：[]，上场棋子[兔子1]；
当前回合还有可用金钱2；
玩家buff有[]；


玩家1 现在拥有 2 人口，升级至下一人口还需要2回合，或者 4 金钱；


当前回合 2, 剩余血量：90
玩家1 现在拥有 2 人口，距离下次升级还需要2回合，或者4金钱；
现在拥有的棋子为{2: 蚂蚁2}
备战区棋子：[]，上场棋子[蚂蚁2]；
当前回合还有可用金钱2；
玩家buff有[]；


玩家0 本次选择有 [<class 'chess_info.littleUglyFish'>, <class 'chess_info.rabbit'>, <class 'chess_info.ant'>]
玩家1 本次选择有 [<class 'chess_info.ant'>, <class 'chess_info.rabbit'>, <class 'chess_info.ant'>]


[chess_info.ant, chess_info.rabbit, chess_info.ant]

In [8]:
player1.get_chess(ant())
player1.get_chess(ant())
player2.get_chess(rabbit())
player2.get_chess(ant())
player1.printStatus()
player2.printStatus()
# 加一个现有棋子的打印

玩家0 购买棋子蚂蚁3 成功,共花费:  1  金币，还剩  1  金币。
玩家0 购买棋子蚂蚁4 成功,共花费:  1  金币，还剩  0  金币。
玩家1 购买棋子兔子5 成功,共花费:  1  金币，还剩  1  金币。
玩家1 购买棋子蚂蚁6 成功,共花费:  1  金币，还剩  0  金币。

当前回合 2, 剩余血量：90
玩家0 现在拥有 2 人口，距离下次升级还需要2回合，或者4金钱；
现在拥有的棋子为{1: 兔子1, 3: 蚂蚁3, 4: 蚂蚁4}
备战区棋子：[蚂蚁3, 蚂蚁4]，上场棋子[兔子1]；
当前回合还有可用金钱0；
玩家buff有[]；



当前回合 2, 剩余血量：90
玩家1 现在拥有 2 人口，距离下次升级还需要2回合，或者4金钱；
现在拥有的棋子为{2: 蚂蚁2, 5: 兔子5, 6: 蚂蚁6}
备战区棋子：[兔子5, 蚂蚁6]，上场棋子[蚂蚁2]；
当前回合还有可用金钱0；
玩家buff有[]；




In [9]:
# print(player1.chesses[3])

player1.upgrade_chess(chessID = 3)

3 <class 'int'> {1: 兔子1, 3: 蚂蚁3, 4: 蚂蚁4}
3 <class 'int'> {1: 兔子1, 3: 蚂蚁3, 4: 蚂蚁4}
  玩家0:蚂蚁3升级成功，请选择一个 2 星棋子
玩家0 通过升级获取了 <瓢虫7>


In [10]:
player2.upgrade_chess(chessID = 2)

2 <class 'int'> {2: 蚂蚁2, 5: 兔子5, 6: 蚂蚁6}
2 <class 'int'> {2: 蚂蚁2, 5: 兔子5, 6: 蚂蚁6}
  玩家1:蚂蚁2升级成功，请选择一个 2 星棋子
玩家1 通过升级获取了 <蜜蜂8>


In [11]:
# 准备阶段

# 上场棋子
player1Chess = 7
player1.hand_to_field(player1Chess, position=[0,0])


player2Chess = 5
player2.hand_to_field(player2Chess, position=[3,2])

player2Chess = 8
player2.hand_to_field(player2Chess, position=[4,2])

# 战斗阶段
round2Battle = battle()
round2Battle.addRedTeam(redTeam= player1.chessOnField)
round2Battle.addBlueTeam(blueTeam= player2.chessOnField)
round2Battle.board_print()
round2Battle.battle_with_skills()


7 <class 'int'> {1: 兔子1, 7: 瓢虫7}
玩家0:瓢虫7上场到位置[0, 0]
       备战区棋子：[]，
       上场棋子[兔子1, 瓢虫7]；
5 <class 'int'> {5: 兔子5, 8: 蜜蜂8}
玩家1:兔子5上场到位置[3, 2]
       备战区棋子：[蜜蜂8]，
       上场棋子[兔子5]；
8 <class 'int'> {5: 兔子5, 8: 蜜蜂8}
玩家1:蜜蜂8上场到位置[4, 2]
       备战区棋子：[]，
       上场棋子[兔子5, 蜜蜂8]；

[瓢虫7, None, None, None, None]

[None, None, None, None, None]

[None, None, 兔子1, None, None]

[None, None, 兔子5, None, None]

[None, None, 蜜蜂8, None, None]

[None, None, None, None, None]



[瓢虫7, None, None, None, None]

[None, None, None, None, None]

[None, None, 兔子1, None, None]

[None, None, 兔子5, None, None]

[None, None, 蜜蜂8, None, None]

[None, None, None, None, None]


0.05   瓢虫7开始从[0, 0]移动到[3, 2] 
0.05 <瓢虫7>向<down> 移动了一格

[None, None, None, None, None]

[瓢虫7, None, None, None, None]

[None, None, 兔子1, None, None]

[None, None, 兔子5, None, None]

[None, None, 蜜蜂8, None, None]

[None, None, None, None, None]


0.05   蜜蜂8开始从[4, 2]移动到[2, 2] 
0.05 <蜜蜂8>向<left> 移动了一格

[None, None, None, None, None]

[瓢虫7, None, Non

回合3


In [13]:
player1.next_round()
player2.next_round()

# player1.printStatus()
# player2.printStatus()

player1.get_random_chess_to_draw()
player2.get_random_chess_to_draw()



当前回合 3, 剩余血量：80
玩家0 现在拥有 2 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{1: 兔子1, 7: 瓢虫7}
备战区棋子：[]，上场棋子[兔子1, 瓢虫7]；
当前回合还有可用金钱3；
玩家buff有[]；



当前回合 3, 剩余血量：80
玩家1 现在拥有 2 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{5: 兔子5, 8: 蜜蜂8}
备战区棋子：[]，上场棋子[兔子5, 蜜蜂8]；
当前回合还有可用金钱3；
玩家buff有[]；


玩家0 本次选择有 [<class 'chess_info.rabbit'>, <class 'chess_info.littleUglyFish'>, <class 'chess_info.ant'>]
玩家1 本次选择有 [<class 'chess_info.rabbit'>, <class 'chess_info.rabbit'>, <class 'chess_info.rabbit'>]


[chess_info.rabbit, chess_info.rabbit, chess_info.rabbit]

In [14]:
player1.get_chess(littleUglyFish())
player1.get_chess(littleUglyFish())
player1.get_chess(ant())
player1.printStatus()


玩家0 购买棋子小丑鱼9 成功,共花费:  1  金币，还剩  2  金币。
玩家0 购买棋子小丑鱼10 成功,共花费:  1  金币，还剩  1  金币。
玩家0 购买棋子蚂蚁11 成功,共花费:  1  金币，还剩  0  金币。

当前回合 3, 剩余血量：80
玩家0 现在拥有 2 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{1: 兔子1, 7: 瓢虫7, 9: 小丑鱼9, 10: 小丑鱼10, 11: 蚂蚁11}
备战区棋子：[小丑鱼9, 小丑鱼10, 蚂蚁11]，上场棋子[兔子1, 瓢虫7]；
当前回合还有可用金钱0；
玩家buff有[]；




In [15]:
player1.upgrade_chess(chessID=9)

9 <class 'int'> {1: 兔子1, 7: 瓢虫7, 9: 小丑鱼9, 10: 小丑鱼10, 11: 蚂蚁11}
9 <class 'int'> {1: 兔子1, 7: 瓢虫7, 9: 小丑鱼9, 10: 小丑鱼10, 11: 蚂蚁11}
  玩家0:小丑鱼9升级成功，请选择一个 2 星棋子
玩家0 通过升级获取了 <食人鱼12>


In [16]:
player1.printStatus()
player1.field_to_hand(1)
player1.hand_to_field(12,[2,2])


当前回合 3, 剩余血量：80
玩家0 现在拥有 2 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{1: 兔子1, 7: 瓢虫7, 11: 蚂蚁11, 12: 食人鱼12}
备战区棋子：[蚂蚁11, 食人鱼12]，上场棋子[兔子1, 瓢虫7]；
当前回合还有可用金钱0；
玩家buff有[]；


1 <class 'int'> {1: 兔子1, 7: 瓢虫7, 11: 蚂蚁11, 12: 食人鱼12}
玩家0:兔子1下场
       备战区棋子：[蚂蚁11, 食人鱼12, 兔子1]，
       上场棋子[瓢虫7]；
12 <class 'int'> {1: 兔子1, 7: 瓢虫7, 11: 蚂蚁11, 12: 食人鱼12}
玩家0:食人鱼12上场到位置[2, 2]
       备战区棋子：[蚂蚁11, 兔子1]，
       上场棋子[瓢虫7, 食人鱼12]；


In [17]:
player2.get_chess(littleUglyFish())
player2.get_chess(littleUglyFish())
player2.get_chess(rabbit())
player2.printStatus()

玩家1 购买棋子小丑鱼13 成功,共花费:  1  金币，还剩  2  金币。
玩家1 购买棋子小丑鱼14 成功,共花费:  1  金币，还剩  1  金币。
玩家1 购买棋子兔子15 成功,共花费:  1  金币，还剩  0  金币。

当前回合 3, 剩余血量：80
玩家1 现在拥有 2 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{5: 兔子5, 8: 蜜蜂8, 13: 小丑鱼13, 14: 小丑鱼14, 15: 兔子15}
备战区棋子：[小丑鱼13, 小丑鱼14, 兔子15]，上场棋子[兔子5, 蜜蜂8]；
当前回合还有可用金钱0；
玩家buff有[]；




In [18]:
player1.printStatus()
player2.printStatus()



当前回合 3, 剩余血量：80
玩家0 现在拥有 2 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{1: 兔子1, 7: 瓢虫7, 11: 蚂蚁11, 12: 食人鱼12}
备战区棋子：[蚂蚁11, 兔子1]，上场棋子[瓢虫7, 食人鱼12]；
当前回合还有可用金钱0；
玩家buff有[]；



当前回合 3, 剩余血量：80
玩家1 现在拥有 2 人口，距离下次升级还需要1回合，或者1金钱；
现在拥有的棋子为{5: 兔子5, 8: 蜜蜂8, 13: 小丑鱼13, 14: 小丑鱼14, 15: 兔子15}
备战区棋子：[小丑鱼13, 小丑鱼14, 兔子15]，上场棋子[兔子5, 蜜蜂8]；
当前回合还有可用金钱0；
玩家buff有[]；




In [19]:
from battle import battle
# # 战斗阶段

round3Battle = battle()
round3Battle.board_clear()
round3Battle.board_print()

round3Battle.addRedTeam(redTeam= player1.chessOnField)
round3Battle.addBlueTeam(blueTeam= player2.chessOnField)
round3Battle.board_print()
round3Battle.battle_with_skills()


[None, None, None, None, None]

[None, None, None, None, None]

[None, None, None, None, None]

[None, None, None, None, None]

[None, None, None, None, None]

[None, None, None, None, None]



[瓢虫7, None, None, None, None]

[None, None, None, None, None]

[None, None, 食人鱼12, None, None]

[None, None, 兔子5, None, None]

[None, None, 蜜蜂8, None, None]

[None, None, None, None, None]



[瓢虫7, None, None, None, None]

[None, None, None, None, None]

[None, None, 食人鱼12, None, None]

[None, None, 兔子5, None, None]

[None, None, 蜜蜂8, None, None]

[None, None, None, None, None]


0.9 <兔子5> 攻击了<食人鱼12>, 造成了15.000000000000004点伤害，<食人鱼12>生命值还剩:365.0 / 380
1.2 <食人鱼12> 攻击了<兔子5>, 造成了107.97546012269939点伤害，<兔子5>生命值还剩:142.0245398773006 / 250


KeyError: 'dispersion_status'

回合4


In [ ]:
# 回合开始
# player1.next_round()
# player2.next_round()

# player1.get_random_chess_to_draw()
# player2.get_random_chess_to_draw()

# #购买棋子
# player1.get_chess(ant())
# player1.get_chess(ant())
# player2.get_chess(rabbit())
# player2.get_chess(ant())
# player1.printStatus()
# player2.printStatus()
# player1.upgrade_chess(chessID = 3)
# # 上场棋子
# player1Chess = 7
# player1.hand_to_field(player1Chess, position=[0,0])


# player2Chess = 5
# player2.hand_to_field(player2Chess, position=[3,2])

# player2Chess = 8
# player2.hand_to_field(player2Chess, position=[4,2])

# # 战斗阶段
# round2Battle = battle()
# round2Battle.addRedTeam(redTeam= player1.chessOnField)
# round2Battle.addBlueTeam(blueTeam= player2.chessOnField)
# round2Battle.board_print()
# round2Battle.battle_with_skills()

回合5


In [ ]:
# 回合开始
# player1.next_round()
# player2.next_round()

# player1.get_random_chess_to_draw()
# player2.get_random_chess_to_draw()

# #购买棋子
# player1.get_chess(ant())
# player1.get_chess(ant())
# player2.get_chess(rabbit())
# player2.get_chess(ant())
# player1.printStatus()
# player2.printStatus()
# player1.upgrade_chess(chessID = 3)
# # 上场棋子
# player1Chess = 7
# player1.hand_to_field(player1Chess, position=[0,0])


# player2Chess = 5
# player2.hand_to_field(player2Chess, position=[3,2])

# player2Chess = 8
# player2.hand_to_field(player2Chess, position=[4,2])

# # 战斗阶段
# round2Battle = battle()
# round2Battle.addRedTeam(redTeam= player1.chessOnField)
# round2Battle.addBlueTeam(blueTeam= player2.chessOnField)
# round2Battle.board_print()
# round2Battle.battle_with_skills()

回合6


In [ ]:
# 回合开始
# player1.next_round()
# player2.next_round()

# player1.get_random_chess_to_draw()
# player2.get_random_chess_to_draw()

# #购买棋子
# player1.get_chess(ant())
# player1.get_chess(ant())
# player2.get_chess(rabbit())
# player2.get_chess(ant())
# player1.printStatus()
# player2.printStatus()
# player1.upgrade_chess(chessID = 3)
# # 上场棋子
# player1Chess = 7
# player1.hand_to_field(player1Chess, position=[0,0])


# player2Chess = 5
# player2.hand_to_field(player2Chess, position=[3,2])

# player2Chess = 8
# player2.hand_to_field(player2Chess, position=[4,2])

# # 战斗阶段
# round2Battle = battle()
# round2Battle.addRedTeam(redTeam= player1.chessOnField)
# round2Battle.addBlueTeam(blueTeam= player2.chessOnField)
# round2Battle.board_print()
# round2Battle.battle_with_skills()

回合7


In [ ]:
# 回合开始
# player1.next_round()
# player2.next_round()

# player1.get_random_chess_to_draw()
# player2.get_random_chess_to_draw()

# #购买棋子
# player1.get_chess(ant())
# player1.get_chess(ant())
# player2.get_chess(rabbit())
# player2.get_chess(ant())
# player1.printStatus()
# player2.printStatus()
# player1.upgrade_chess(chessID = 3)
# # 上场棋子
# player1Chess = 7
# player1.hand_to_field(player1Chess, position=[0,0])


# player2Chess = 5
# player2.hand_to_field(player2Chess, position=[3,2])

# player2Chess = 8
# player2.hand_to_field(player2Chess, position=[4,2])

# # 战斗阶段
# round2Battle = battle()
# round2Battle.addRedTeam(redTeam= player1.chessOnField)
# round2Battle.addBlueTeam(blueTeam= player2.chessOnField)
# round2Battle.board_print()
# round2Battle.battle_with_skills()

In [ ]:
# 回合开始
# player1.next_round()
# player2.next_round()

# player1.get_random_chess_to_draw()
# player2.get_random_chess_to_draw()

# #购买棋子
# player1.get_chess(ant())
# player1.get_chess(ant())
# player2.get_chess(rabbit())
# player2.get_chess(ant())
# player1.printStatus()
# player2.printStatus()
# player1.upgrade_chess(chessID = 3)
# # 上场棋子
# player1Chess = 7
# player1.hand_to_field(player1Chess, position=[0,0])


# player2Chess = 5
# player2.hand_to_field(player2Chess, position=[3,2])

# player2Chess = 8
# player2.hand_to_field(player2Chess, position=[4,2])

# # 战斗阶段
# round2Battle = battle()
# round2Battle.addRedTeam(redTeam= player1.chessOnField)
# round2Battle.addBlueTeam(blueTeam= player2.chessOnField)
# round2Battle.board_print()
# round2Battle.battle_with_skills()

In [ ]:
# 回合开始
# player1.next_round()
# player2.next_round()

# player1.get_random_chess_to_draw()
# player2.get_random_chess_to_draw()

# #购买棋子
# player1.get_chess(ant())
# player1.get_chess(ant())
# player2.get_chess(rabbit())
# player2.get_chess(ant())
# player1.printStatus()
# player2.printStatus()
# player1.upgrade_chess(chessID = 3)
# # 上场棋子
# player1Chess = 7
# player1.hand_to_field(player1Chess, position=[0,0])


# player2Chess = 5
# player2.hand_to_field(player2Chess, position=[3,2])

# player2Chess = 8
# player2.hand_to_field(player2Chess, position=[4,2])

# # 战斗阶段
# round2Battle = battle()
# round2Battle.addRedTeam(redTeam= player1.chessOnField)
# round2Battle.addBlueTeam(blueTeam= player2.chessOnField)
# round2Battle.board_print()
# round2Battle.battle_with_skills()

In [ ]:
# 回合开始
# player1.next_round()
# player2.next_round()

# player1.get_random_chess_to_draw()
# player2.get_random_chess_to_draw()

# #购买棋子
# player1.get_chess(ant())
# player1.get_chess(ant())
# player2.get_chess(rabbit())
# player2.get_chess(ant())
# player1.printStatus()
# player2.printStatus()
# player1.upgrade_chess(chessID = 3)
# # 上场棋子
# player1Chess = 7
# player1.hand_to_field(player1Chess, position=[0,0])


# player2Chess = 5
# player2.hand_to_field(player2Chess, position=[3,2])

# player2Chess = 8
# player2.hand_to_field(player2Chess, position=[4,2])

# # 战斗阶段
# round2Battle = battle()
# round2Battle.addRedTeam(redTeam= player1.chessOnField)
# round2Battle.addBlueTeam(blueTeam= player2.chessOnField)
# round2Battle.board_print()
# round2Battle.battle_with_skills()

In [ ]:
    # [ [0,0] [0,1] [0,2] [0,3] [0,4] ]
    # [ [1,0] [1,1] [1,2] [1,3] [1,4] ]
    # [ [2,0] [2,1] [2,2] [2,3] [2,4] ]
    
    # [ [3,0] [3,1] [3,2] [3,3] [3,4] ]
    # [ [4,0] [4,1] [4,2] [4,3] [4,4] ]
    # [ [5,0] [5,1] [5,2] [5,3] [5,4] ]
# blueTeam = [bear(position = [3,2]), bee(position=[5,4]),sea_hedgehog(position=[3,4])]